In [20]:
from bs4 import BeautifulSoup
import requests
import time


from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 


In [34]:
# driver = webdriver.Chrome()
# driver.get('https://docs.foursquare.com/data-products/docs/categories')
# time.sleep(3)
# driver.find_element("xpath", '//*[@id="onetrust-reject-all-handler"]').click()
# cat_frame = driver.find_element("xpath", '//*[@id="content-container"]/section[1]/div[1]/div[1]/iframe')
# driver.switch_to.frame(cat_frame)
# table_html = driver.find_element("xpath", '/html/body/div/div[1]/div/div/div/table/tbody')